In [1]:
!python --version

Python 3.7.10


In [2]:
pip install tflearn

In [3]:
pip install tensorflow==1.14.0

In [4]:
!pip install flask-ngrok

In [5]:
from IPython.display import Javascript

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import tensorflow as tf
import tflearn
import numpy as np
import random
import json
import nltk
import pickle
import math
import binascii
import webbrowser
import urllib.error
import urllib.request
import smtplib
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import time
import os
import sys
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
tf.version.VERSION

'1.14.0'

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
qaCode = binascii.unhexlify("5468616e6b20796f7520666f722070617274696369706174696e6720666f72203130206d696e75746573212054686520436f646520666f7220746865207175657374696f6e6e61697265206973204135337274593133322120596f752063616e206b6565702074657374696e6720696620796f752077616e743a203c62722f3e3c62722f3e")

In [11]:
try:
  with open('intents.json') as json_data:
      intents = json.load(json_data)
except FileNotFoundError:
  print("json file not found")
  sys.exit('Closing Program')

In [12]:
files = ["./templates/index.html", "./static/pop.mp3", "./static/css/style.css"]
for item in files:
  if os.path.isfile(item):
    continue
  else:
    print(item + " does not exist in the correct directory! Please refer back to the guidelines found in the email")
    sys.exit('Closing Program')

In [13]:
words = []
classes = []
documents = []

In [14]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        try:
          w = nltk.word_tokenize(pattern)
        except LookupError as ntlkerror:
          print("Error during tokenization: ", ntlkerror)
          sys.exit('Closing Program')

        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [15]:
# stem and lower each word and remove duplicates
try:
  words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
  words = sorted(list(set(words)))
except IndexError:
  print("Error stemming words")
  sys.exit('Closing Program')

In [16]:
# remove duplicates
classes = sorted(list(set(classes)))

In [17]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [18]:
# training set, bag of words for each sentence
for doc in documents:
    bag_of_words = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # bag_of_words array
    for w in words:
      if w in pattern_words:
        bag_of_words.append(1)
      else:
        bag_of_words.append(0)


    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag_of_words, output_row])

In [19]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [20]:
# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [21]:
try:
  tf.reset_default_graph()
except AttributeError as tfgrapherror:
  print("Error resetting tensorflow graph: ", tfgrapherror)
  sys.exit('Closing Program')

In [22]:
try:
  hiddenLayerNode = math.ceil(((len(train_x[0]) + len(train_y[0])) / 2))
  net = tflearn.input_data(shape=[None, len(train_x[0])])
  net = tflearn.fully_connected(net, hiddenLayerNode)
  net = tflearn.fully_connected(net, math.ceil((hiddenLayerNode / 2)))
  net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
  net = tflearn.regression(net)
except ValueError as nnbuilderror:
  print("Error creating the neural network model: ", nnbuilderror)
  sys.exit('Closing Program')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
if not os.path.isdir('./tflearnModel'):
    try:
        os.mkdir("./tflearnModel")
    except OSError:
        print("Creation of the directory failed")
        sys.exit('Closing Program')
    else:
        print("Successfully created the directory")
        

In [24]:
# Define model and setup tensorboard
try:
  model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
  # Start training (apply gradient descent algorithm)
  model.fit(train_x, train_y, n_epoch=100, batch_size=32, show_metric=True)
  model.save('./tflearnModel/model.tflearn')
except IndexError as nnfiterror:
  print("Error fitting the neural network model: ", nnfiterror)
  sys.exit('Closing Program')

Training Step: 2499  | total loss: 0.00236 | time: 0.131s
| Adam | epoch: 100 | loss: 0.00236 - acc: 0.9979 -- iter: 768/779
Training Step: 2500  | total loss: 0.00221 | time: 0.138s
| Adam | epoch: 100 | loss: 0.00221 - acc: 0.9981 -- iter: 779/779
--
INFO:tensorflow:/content/tflearnModel/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [25]:
try:
  pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y},
              open("./tflearnModel/model.tflearn", "wb"))
except (IOError, OSError, pickle.PickleError, pickle.UnpicklingError):
  print("Error writing/saving pickled data as tflearn model")
  sys.exit("Closing Program")
except FileNotFoundError:
  print("tflearn model does not exist")
  sys.exit("Closing Program")

In [26]:
try:
  data = pickle.load(open("./tflearnModel/model.tflearn", "rb"))
except (IOError, OSError, pickle.PickleError, pickle.UnpicklingError):
  print("Error loading pickled data from tflearn model")
  sys.exit("Closing Program")
except FileNotFoundError:
  print("tflearn model does not exist")
  sys.exit("Closing Program")

words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [27]:
class ContextHistory:
    contextOld = ""
    myIntent = ""
    myOldTag = ""
    myTag = ""
    myFirstContext = ""

    myFirstInput = 0

    myEmailIntent = 0
    studentEmail = ""
    studentPassword = ""
    staffEmail = ""
    symptomType = ""
    symptomProblem = ""

    time = 0.0
    startTime = 0.0

    chatRestart = "<br/><br/>Please specify if you want " \
                  "guidance for your <fears> or anxiety <symptoms> whilst at Cardiff Met during the coronavirus " \
                  "epidemic. Input (fears) or (symptoms)"
myOldContext = ContextHistory()

In [28]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [29]:
model.load('./tflearnModel/model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/tflearnModel/model.tflearn


In [30]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [31]:
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag_of_words = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag_of_words[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return np.array(bag_of_words)

In [32]:
def myTime():
    if ContextHistory.myFirstInput < 1:
        ContextHistory.startTime = time.perf_counter()

    ContextHistory.time = (time.perf_counter() - ContextHistory.startTime)
    print(ContextHistory.time)

In [33]:
# create a data structure to hold user context
context = {}

In [34]:
ERROR_THRESHOLD = 0.70

In [35]:
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [36]:
def filtering(i, show_details):
    if 'context_set' in i:
        if show_details:
            return 'context:', i['context_set']
        myOldContext.contextOld = i['context_set']
        myOldContext.myTag = i['tag']
        return i['context_set']

In [37]:
def advancedResponse(i):
    if 'web' in i:
        myWeb = i['web']
        try:
            urllib.request.urlopen(myWeb)
        except urllib.error.HTTPError:
            return "I am sorry but I am having trouble connecting to the site: " + random.choice(i['responses'])
        except urllib.error.URLError:
            return "I am sorry but I am having trouble connecting to the site" + random.choice(i['responses'])
        else:
            url = myWeb
            display(Javascript('window.open("{url}");'.format(url=url)))
            return "Redirecting: " + random.choice(i['responses'])

    else:
        return random.choice(i['responses'])

In [38]:
def email(userInput):
    if myOldContext.myEmailIntent == 1:
        myOldContext.studentEmail = str(userInput) + "@outlook.cardiffmet.ac.uk"
        print(str(userInput) + "from email")
        myOldContext.myEmailIntent += 1
        return "Please Enter Your Student Password"
    else:
        recEmail = myOldContext.staffEmail
        subject = myOldContext.studentEmail + " " + myOldContext.symptomType + " anxiety symptom issues"
        body = "Dear Sir/Madam,\n\n This student has been recorded at a high priority as the participant has shown " \
               "signs of suffering from severe " + myOldContext.symptomType + " anxiety symptoms. The symptoms " \
                                                                              "specified indicate that the " \
                                                                              "student could be suffering from " \
                                                                              "anxities that can lead to " + \
               myOldContext.symptomProblem + "\n\nPlease Email them as soon as possible as to arrange a formal meetup " \
                                             "and/or conversation. \n\n Kindest Regards\nMetMind Chatbot"
        msg = f'Subject: {subject}\n\n{body}'

        myOldContext.studentPassword = str(userInput)

        try:
            server = smtplib.SMTP('smtp.outlook.com', 587)
            server.starttls()
            server.login(myOldContext.studentEmail, myOldContext.studentPassword)
            server.sendmail(myOldContext.studentEmail, recEmail, msg)
            myOldContext.myEmailIntent = 0
            myOldContext.contextOld = ""

            myOldContext.myIntent = ""
            myOldContext.myOldTag = myOldContext.chatRestart
            return "email sent! you will hear from the wellbeing team shortly" + myOldContext.chatRestart
        except smtplib.SMTPAuthenticationError:
            myOldContext.myEmailIntent = 1
            return "Incorrect email or password: please enter your username"

In [39]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        print(results)
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    if 'email' in i:
                        myOldContext.myEmailIntent = 1
                        myOldContext.staffEmail = i['email']
                        myOldContext.symptomType = i['symptom_type']
                        myOldContext.symptomProblem = i['symptom_problem']

                    if 'context_set' in i and 'context_filter' not in i and myOldContext.myFirstContext == "":
                        myOldContext.myOldTag = random.choice(i['responses'])
                        myOldContext.myFirstContext = i['context_set']
                        myOldContext.myIntent = i['intent']

                    try:
                        if ('intent' in i and i['context_filter'] == myOldContext.contextOld) or (i['tag'] == myOldContext.myTag):
                            myOldContext.myOldTag = random.choice(i['responses'])
                            myOldContext.myIntent = i['intent']
                            print(myOldContext.myIntent)

                    except KeyError:
                        print("Filter Not In Requested Object")

                    
                    print(myOldContext.myOldTag)
                    print(myOldContext.contextOld)

                    if 'context_filter' in i and i['context_filter'] == myOldContext.contextOld:
                        context[userID] = filtering(i, show_details)
                        if 'context_set' not in i and myOldContext.myEmailIntent < 1:
                            myOldContext.contextOld = ""
                            myOldContext.myIntent = ""
                            myOldContext.myFirstContext = ""
                            myOldContext.myOldTag = myOldContext.chatRestart
                            myResp = advancedResponse(i) + myOldContext.chatRestart
                        else:
                            myResp = advancedResponse(i)
                        print("reached")
                        return myResp
                    elif 'context_filter' not in i and myOldContext.contextOld != "" or (
                            'context_filter' in i and i['context_filter'] != myOldContext.contextOld):
                        return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag
                    else:
                        context[userID] = filtering(i, show_details)

                        if 'context_filter' not in i or \
                                (userID in context and 'context_filter' in i and i['context_filter'] == context[
                                    userID]):
                            if show_details:
                                return 'tag:', i['tag']
                            # a random response from the intent
                            return random.choice(i['responses'])

            results.pop(0)
    else:
        if myOldContext.myIntent != "":
            return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag
        else:
            return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag

In [40]:
app = Flask(__name__)
run_with_ngrok(app)

In [41]:
@app.route('/get')
def userChat():
    myStr = ""
    if ContextHistory.time > 600.00 and ContextHistory.myFirstInput != 2:
        ContextHistory.myFirstInput = 2

        myStr += qaCode.decode()


    userInput = request.args.get('msg')

    myTime()

    if myOldContext.myEmailIntent != 0:
        myStr = email(userInput)
    else:
        if myOldContext.myIntent != "":
            userInput = myOldContext.myIntent + userInput
        print(userInput)
        myStr += response(userInput)

        if ContextHistory.myFirstInput < 1:
            ContextHistory.myFirstInput = 1
    return str(myStr)

In [42]:
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0ff5728bf073.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Apr/2021 15:54:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 15:54:30] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 15:54:30] "GET /static/pop.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [04/Apr/2021 15:54:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 15:54:33] "GET /get?msg=fears HTTP/1.1" 200 -


5.38099993718788e-06
fears
[('fearGroup', 0.9943064)]
Filter Not In Requested Object
Please specify what you are most fearful of whilst at university during the Covid-19 Epidemic! I.E. 'I am scared that the new block system will affect my grades'



...

12.878699638999933
fears
[('fearGroup', 0.9943064)]
Filter Not In Requested Object
Please specify what you are most fearful of whilst at university during the Covid-19 Epidemic! I.E. 'I am scared that the new block system will affect my grades'



127.0.0.1 - - [04/Apr/2021 15:54:48] "GET /get?msg=hgfghffgh HTTP/1.1" 200 -


14.796259886999906
usrChoiceTwoGroup_hgfghffgh


127.0.0.1 - - [04/Apr/2021 15:56:19] "GET /get?msg=symptom HTTP/1.1" 200 -


105.96691388799991
usrChoiceTwoGroup_symptom
